In [1]:
import pandas as pd
import os
import datetime
import shutil
from PIL import Image

In [2]:
df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)
df

C:\Users\elise\AppData\Local\Temp\ipykernel_40876\1146341607.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)


,image_name,x_1,y_1,x_2,y_2
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262
...,...,...,...,...,...
289217,img/Paisley_Print_Babydoll_Dress/img_00000050.jpg,1,1,300,300
289218,img/Paisley_Print_Babydoll_Dress/img_00000051.jpg,14,58,225,277
289219,img/Paisley_Print_Babydoll_Dress/img_00000052.jpg,18,41,149,230
289220,img/Paisley_Print_Babydoll_Dress/img_00000053.jpg,75,47,220,300


In [3]:
category_df = pd.read_csv("labels/Anno_coarse/categories.txt", delimiter=",", header=0)
category_df

,category_name,category_type
0,Anorak,0
1,Blazer,1
2,Blouse,2
3,Bomber,3
4,Button-Down,4
5,Cardigan,5
6,Flannel,6
7,Halter,7
8,Henley,8
9,Hoodie,9


In [4]:
def corr_positions(df, cat_num):
    '''Corriger les positions des boîtes englobantes pour les coordonnées des centres et les dimensions des boîtes englobantes
    df : DataFrame des positions des boîtes englobantes
    cat_num : int
    return : DataFrame'''
    
    corr_data = []   
    
    for row in df.itertuples():
        img_path = 'data/' + row.image_name
        with open(img_path, 'rb') as img_file:
            image = Image.open(img_file)
            img_width, img_height = image.size
        image = row.image_name.replace('img/', '')
        img_name = image.replace('/', '_')
        width = (row.x_2 - row.x_1) 
        height = (row.y_2 - row.y_1)
        x_center = ((width/2) + row.x_1) / img_width
        y_center = ((height/2) + row.y_1) / img_height
        width = width / img_width
        height = height / img_height
        corr_data.append({'image_name': img_name, 'category_type': cat_num, 'x_center': x_center, 'y_center': y_center, 'width': width, 'height': height}) 
    corr_df = pd.DataFrame(corr_data)
    return corr_df

In [5]:
def find_category(df, category_name):
    '''Trouver la catégorie de vêtement dans le DataFrame
    df : DataFrame des catégories
    category_name : str
    return : int'''
    
    return df[df['category_name'].str.contains(category_name)].category_type.values[0]

Tests : 

In [6]:
blouse_df = df[df['image_name'].str.contains('Blouse')]
blouse_df

,image_name,x_1,y_1,x_2,y_2
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262
...,...,...,...,...,...
32846,img/Sheer_Lace_&_Chiffon_Blouse/img_00000130.jpg,73,77,182,279
32847,img/Sheer_Lace_&_Chiffon_Blouse/img_00000131.jpg,1,7,178,300
32848,img/Sheer_Lace_&_Chiffon_Blouse/img_00000132.jpg,42,69,205,267
32849,img/Sheer_Lace_&_Chiffon_Blouse/img_00000133.jpg,73,31,300,300


In [7]:
blouse_cat = find_category(category_df, 'Blouse')
blouse_corr_df = corr_positions(blouse_df, blouse_cat)
blouse_corr_df.head()

,image_name,category_type,x_center,y_center,width,height
0,Sheer_Pleated-Front_Blouse_img_00000001.jpg,2,0.506667,0.586667,0.533333,0.646667
1,Sheer_Pleated-Front_Blouse_img_00000002.jpg,2,0.541463,0.366667,0.429268,0.340000
2,Sheer_Pleated-Front_Blouse_img_00000003.jpg,2,0.533816,0.441667,0.439614,0.450000
3,Sheer_Pleated-Front_Blouse_img_00000004.jpg,2,0.545000,0.488000,0.580000,0.480000
4,Sheer_Pleated-Front_Blouse_img_00000005.jpg,2,0.530000,0.583333,0.600000,0.580000


Fonctions pour créer le dossier utilisé pour l'entraînement : copier les images et modifier les labels nécessaires :

In [8]:
def creer_fichier(chemin_fichier):
    '''Créer les dossier et fichiers nécessaires pour le fichier txt ou les images
    chemin_fichier : str'''

    dossier = os.path.dirname(chemin_fichier)
    if not os.path.exists(dossier):
        os.makedirs(dossier)

In [9]:
def modification_date(chemin_fichier):
    '''Trouver la date de la dernière modification d'un fichier
    chemin_fichier : str
    return : datetime'''
    
    modification_timestamp = os.path.getmtime(chemin_fichier)
    modification_date = datetime.datetime.fromtimestamp(modification_timestamp)
    return modification_date

In [10]:
def creer_txt(df, row, chemin_fichier):
    '''Créer un fichier txt avec les coordonnées des boîtes englobantes
    df : DataFrame des bonnes coordonnées des boîtes englobantes (cat, x_center, y_center, width, height)
    row : Series
    chemin_fichier : str'''

    ligne = df.iloc[row.Index]
    ligne = ligne.drop(['image_name'])
    ligne_concatenee = ' '.join(map(str, ligne))
    with open(chemin_fichier, 'a+') as fichier:
        fichier.seek(0)  # Retourner au début du fichier
        contenu_existant = fichier.read()  # Lire le contenu existant du fichier
        if modification_date(chemin_fichier) < (datetime.datetime.now() - datetime.timedelta(seconds=60)):  # Si le fichier a été modifié pour la dernière fois il y a plus de 60 secondes, le vider
            fichier.truncate(0)
        elif contenu_existant:  # Sinon si le fichier n'est pas vide ajouter une nouvelle ligne
            fichier.write('\n')  # Cas utile lorsqu'il y a plusieurs vetements sur la meme image
        fichier.write(ligne_concatenee)

In [11]:
def chemins(row):
    '''Mettre en forme le chemin du fichier txt depuis le DataFrame des positions des boîtes englobantes (qui possède le nom des images)
    row : Series
    return : str'''
    
    chemin_img = 'otherdata/images/' + row.image_name
    chemin_label = 'otherdata/labels/' + row.image_name.replace('.jpg', '') + '.txt'
    return chemin_img, chemin_label

In [17]:
def copie_images(df, row, chemin_img_arrivee):
    '''Copier les images dans le dossier de destination des images et labels à utiliser pour l'entraînement
    df : DataFrame des positions des boîtes englobantes
    category_df : DataFrame des catégories
    data_folder : str'''
    
    chemin_img_depart = 'data/' + df.iloc[row.Index].image_name
    shutil.copy2(chemin_img_depart, chemin_img_arrivee)

In [18]:
def prepare_data(df, category_df, category):
    '''Scinder les labels des boîtes englobantes pour chaque catégorie de vêtement
    Il faut un label (fichier txt) par image
    df : DataFrame des positions des boîtes englobantes
    category_df : DataFrame des catégories
    category : str'''

    cat = find_category(category_df, category)
    df_cat = df[df['image_name'].str.contains(category)]
    df_corr = corr_positions(df, cat)
    for row in df_corr.itertuples():
        chemin_img, chemin_label = chemins(row)
        creer_fichier(chemin_label)
        creer_fichier(chemin_img)
        creer_txt(df_corr, row, chemin_label)
        copie_images(df_cat, row, chemin_img)

Préparer tout le dataset pour l'entraînement:

In [14]:
def prepare_data_toutes_categories(df, category_df):
    '''Scinder les labels des boîtes englobantes pour toutes les catégories de vêtements
    df : DataFrame des positions des boîtes englobantes'''
    
    for row in category_df.itertuples():
        prepare_data(df, category_df, row.category_name)

In [15]:
#prepare_data_toutes_categories(df, category_df)

Tests :

In [21]:
prepare_data(blouse_df, category_df, 'Blouse')

Création du fichier de configuration YAML pour YOLOv8 :

In [ ]:
def create_yaml_file(yaml_file, category_df):
    with open(yaml_file, 'w') as file:
        file.write('path: ' + os.getcwd() + '\\otherdata\n')
        file.write('train: images\n')
        file.write('val: images\n')
        file.write('names:\n')
        for row in category_df.itertuples():
            file.write('    ' + str(row.category_type) + ': ' + row.category_name + '\n')

In [ ]:
create_yaml_file('config.yaml', category_df)

Vérification des labels :

In [ ]:
def check_labels(data_folder):
    '''Vérifier si les annotations des boîtes englobantes sont correctes
    data_folder : str'''

    image_folder = os.path.join(data_folder, 'images')
    label_folder = os.path.join(data_folder, 'labels')

    images = set(os.listdir(image_folder))
    labels = set(os.listdir(label_folder))

    missing_labels = [img.replace('.jpg', '.txt') for img in images if img.replace('.jpg', '.txt') not in labels]
    print(f'Missing labels for images: {missing_labels}')

    for label_file in labels:
        path = os.path.join(label_folder, label_file)
        with open(path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Error in {label_file}: {line.strip()}")


In [ ]:
check_labels('otherdata')